# Knowledge Mapper

To create a embedding visualization based on text data from Unit handbook that helps students decide which courses to pursue based on their interests and current enrolled units.

```
Key fields:
1. UNIT_TITLE
2. HANDBOOK_SYNOPSIS
3. UNIT_LEARNING_OUTCOME

Unit meta data:
1. UNIT_CODE
2. ABBREVIATED_UNIT_TITLE
Colour by:
1. STUDY_LEVEL (UG vs. PG)
2. OWNING_FACULTY (faculty who teaches it)
3. OWNING_ORG_UNIT (department who teaches it)

Auxillary questions:
1. What other units should be prohibited to be taken in dual with this unit?
2. What units should I take before this unit? 

Note: PUBLISH_TO_HANDBOOK probably should be used as a first-order filter.
```

### [Embedding Projector](https://projector.tensorflow.org/)

Data format:
```
Load data from your computer
Step 1: Load a TSV file of vectors.
Example of 3 vectors with dimension 4:
0.1\t0.2\t0.5\t0.9
0.2\t0.1\t5.0\t0.2
0.4\t0.1\t7.0\t0.8
Step 2 (optional): Load a TSV file of metadata.
Example of 3 data points and 2 columns.
Note: If there is more than one column, the first row will be parsed as column labels.
Pokémon\tSpecies
Wartortle\tTurtle
Venusaur\tSeed
Charmeleon\tFlame
```

Metadata
```
UNIT_CODE\tUNIT_TITLE\tHANDBOOK_SYNOPSIS\tUNIT_LEARNING_OUTCOME\tSTUDY_LEVEL\tOWNING_FACULTY\tOWNING_ORG_UNIT
```

### Libraries

In [1]:
!pip install transformers

     |████████████████████████████████| 1.9MB 6.5MB/s 
     |████████████████████████████████| 890kB 20.8MB/s 
     |████████████████████████████████| 3.2MB 34.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=5667da9ecb99645101428023d15248e329bb17246d9663d8df81fad329b455d2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!pip install sentence-transformers

     |████████████████████████████████| 71kB 4.0MB/s 
     |████████████████████████████████| 1.2MB 8.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp37-none-any.whl size=103068 sha256=5b9bb4be4fa4201180b079171cfccedeb6f6af24ebe2439aeddc9e34a7ec74ba
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
Successfully built sentence-transformers


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# general
import json
import numpy as np
import pandas as pd
from pathlib import Path

In [ ]:
from sentence_transformers import SentenceTransformer

In [23]:
data_path = Path('/content/drive/MyDrive/knowledge-mapper/data/raw_data/units owned by faculty - one line per unit (as at 18 Feb 2021).xlsx')
df = pd.read_excel(data_path, sheet_name='Sheet1')

In [24]:
df.shape

(5848, 36)

In [25]:
df.columns

Index(['CL_UNIT_ID', 'UNIT_CODE', 'CL_UNIT_VERSION', 'UNIT_TITLE',
       'ABBREVIATED_UNIT_TITLE', 'CREDIT_POINTS', 'UNIT_STATUS',
       'OWNING_FACULTY', 'OWNING_ORG_UNIT', 'HIGHEST_SCA_BAND',
       'IMPLEMENTATION_YR', 'PUBLISH_TO_HANDBOOK', 'STUDY_LEVEL',
       'HIGHEST_SCA_BAND_1', 'UNIT_EFTSL', 'HANDBOOK_SYNOPSIS',
       'WORKLOAD_REQUIREMENTS', 'QUOTA_INFORMATION', 'OTHER_UNIT_COSTS',
       'FIELD_WORK', 'AREA_OF_STUDY_LINKS', 'OFF_CAMPUS_ATTEND_REQUIREMENTS',
       'SPECIAL_NOTE_TO_STUDENTS', 'UNITOFFERING',
       'HANDBOOK_ASSESSMENT_SUMMARY', 'ASSES_ITEMS', 'UNITCOORD', 'CHIEFEXAM',
       'TEACHING_RESPONSIBILITY', 'PREREQUISITE', 'COREQUISITE', 'PROHIBITION',
       'RULES(PREREQ,COREQ,PROH)', 'INFORMATION_RULE', 'LEARNING_OUTCOME_INFO',
       'UNIT_LEARNING_OUTCOME'],
      dtype='object')

In [26]:
df.head()

,CL_UNIT_ID,UNIT_CODE,CL_UNIT_VERSION,UNIT_TITLE,ABBREVIATED_UNIT_TITLE,CREDIT_POINTS,UNIT_STATUS,OWNING_FACULTY,OWNING_ORG_UNIT,HIGHEST_SCA_BAND,IMPLEMENTATION_YR,PUBLISH_TO_HANDBOOK,STUDY_LEVEL,HIGHEST_SCA_BAND_1,UNIT_EFTSL,HANDBOOK_SYNOPSIS,WORKLOAD_REQUIREMENTS,QUOTA_INFORMATION,OTHER_UNIT_COSTS,FIELD_WORK,AREA_OF_STUDY_LINKS,OFF_CAMPUS_ATTEND_REQUIREMENTS,SPECIAL_NOTE_TO_STUDENTS,UNITOFFERING,HANDBOOK_ASSESSMENT_SUMMARY,ASSES_ITEMS,UNITCOORD,CHIEFEXAM,TEACHING_RESPONSIBILITY,PREREQUISITE,COREQUISITE,PROHIBITION,"RULES(PREREQ,COREQ,PROH)",INFORMATION_RULE,LEARNING_OUTCOME_INFO,UNIT_LEARNING_OUTCOME
0,554c86a41b5aac10653b206b274bcbf9,ACB1020,2021.04RO,Accounting in business,ACC IN BUS,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,This unit introduces basic accounting concepts...,Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S1-01-PENINSULA-ON-CAMPUS Offered-Y,NaN,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr Mahendra Goyal,Responsible teaching Department of Accounting ...,NaN,NaN,ACB1120 OR ACF1100 OR ACF1200 OR ACC1100 OR AC...,Prohibition: ACB1120 OR ACF1100 OR ACF1200 OR ...,NaN,"On successful completion of this unit, you sho...",ULO1 - demonstrate an understanding of various...
1,1928deab1bd65c504c45bbbbdc4bcb3a,ACB1100,2021.01RO,Introduction to financial accounting,INTRO FIN ACC,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 3,2021,N,undergraduate,SCA Band 3,0.125,This unit provides students with an introducti...,Minimum total expected workload to achieve the...,NaN,Costs are indicative and subject to change.Ele...,NaN,NaN,NaN,NaN,NaN,Within semester assessment: 50% + Examination:...,NaN,NaN,Dr Mahendra Goyal,Responsible teaching Department of Accounting ...,NaN,NaN,ACC1100 OR ACF1100 OR ACW1100,Prohibition: ACC1100 OR ACF1100 OR ACW1100,NaN,The learning outcomes associated with this uni...,ULO1 - Identify and analyse measurement system...
2,e14cc6a41b5aac10653b206b274bcb3e,ACB1120,2021.04RO,Financial accounting 1,INTRO FIN ACC,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,This unit provides you with an introduction to...,Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S1-01-PENINSULA-ON-CAMPUS Offered-Y,NaN,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr Mahendra Goyal,Responsible teaching Department of Accounting ...,NaN,NaN,ACC1100 OR ACF1100 OR ACW1120,Prohibition: ACC1100 OR ACF1100 OR ACW1120,NaN,"On successful completion of this unit, you sho...",ULO1 - identify and analyse measurement system...
3,a128deab1bd65c504c45bbbbdc4bcbb4,ACB1200,2021.01RO,Accounting for managers,ACC FOR MNGRS,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 3,2021,N,undergraduate,SCA Band 3,0.125,This unit introduces basic accounting concepts...,Minimum total expected workload to achieve the...,NaN,Costs are indicative and subject to change.Ele...,NaN,NaN,NaN,NaN,NaN,Within semester assessment: 50% + Examination:...,NaN,NaN,Mr Jonathan Phillips,Responsible teaching Department of Accounting ...,NaN,NaN,ACF1100 OR ACW1100 OR ACB1100 OR ACC1200 OR AC...,Prohibition: ACF1100 OR ACW1100 OR ACB1100 OR ...,NaN,The learning outcomes associated with this uni...,ULO1 - Ddemonstrate an understanding of variou...
4,b528deab1bd65c504c45bbbbdc4bcbc7,ACB2020,2021.01RO,Cost information for decision making,COST INFO FOR DEC MA,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 3,2021,N,undergraduate,SCA Band 3,0.125,Introduction to management accounting. Topics ...,Minimum total expected workload to achieve the...,NaN,Costs are indicative and subject to change.Ele...,NaN,NaN,NaN,NaN,NaN,Within semester assessment: 50% + Examination:...,NaN,NaN,Mr Paul Yap,Responsible teaching Department of Accounting ...,ACB1100,NaN,NaN,Prerequisite:

### Unique Identifiers

In [27]:
df.CL_UNIT_ID.nunique(), df.UNIT_CODE.nunique()

(5848, 5848)

In [28]:
key_cols = ["CL_UNIT_ID",
            "UNIT_CODE",
            "UNIT_TITLE",
            "HANDBOOK_SYNOPSIS",
            "UNIT_LEARNING_OUTCOME",
            "STUDY_LEVEL",
            "OWNING_FACULTY",
            "OWNING_ORG_UNIT"
            ]

### Filter by `PUBLISH_TO_HANDBOOK`

In [29]:
df = df.loc[df["PUBLISH_TO_HANDBOOK"] == 'Y', :]

In [30]:
df.shape

(5324, 36)

In [31]:
df.head()

,CL_UNIT_ID,UNIT_CODE,CL_UNIT_VERSION,UNIT_TITLE,ABBREVIATED_UNIT_TITLE,CREDIT_POINTS,UNIT_STATUS,OWNING_FACULTY,OWNING_ORG_UNIT,HIGHEST_SCA_BAND,IMPLEMENTATION_YR,PUBLISH_TO_HANDBOOK,STUDY_LEVEL,HIGHEST_SCA_BAND_1,UNIT_EFTSL,HANDBOOK_SYNOPSIS,WORKLOAD_REQUIREMENTS,QUOTA_INFORMATION,OTHER_UNIT_COSTS,FIELD_WORK,AREA_OF_STUDY_LINKS,OFF_CAMPUS_ATTEND_REQUIREMENTS,SPECIAL_NOTE_TO_STUDENTS,UNITOFFERING,HANDBOOK_ASSESSMENT_SUMMARY,ASSES_ITEMS,UNITCOORD,CHIEFEXAM,TEACHING_RESPONSIBILITY,PREREQUISITE,COREQUISITE,PROHIBITION,"RULES(PREREQ,COREQ,PROH)",INFORMATION_RULE,LEARNING_OUTCOME_INFO,UNIT_LEARNING_OUTCOME
0,554c86a41b5aac10653b206b274bcbf9,ACB1020,2021.04RO,Accounting in business,ACC IN BUS,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,This unit introduces basic accounting concepts...,Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S1-01-PENINSULA-ON-CAMPUS Offered-Y,NaN,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr Mahendra Goyal,Responsible teaching Department of Accounting ...,NaN,NaN,ACB1120 OR ACF1100 OR ACF1200 OR ACC1100 OR AC...,Prohibition: ACB1120 OR ACF1100 OR ACF1200 OR ...,NaN,"On successful completion of this unit, you sho...",ULO1 - demonstrate an understanding of various...
2,e14cc6a41b5aac10653b206b274bcb3e,ACB1120,2021.04RO,Financial accounting 1,INTRO FIN ACC,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,This unit provides you with an introduction to...,Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S1-01-PENINSULA-ON-CAMPUS Offered-Y,NaN,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr Mahendra Goyal,Responsible teaching Department of Accounting ...,NaN,NaN,ACC1100 OR ACF1100 OR ACW1120,Prohibition: ACC1100 OR ACF1100 OR ACW1120,NaN,"On successful completion of this unit, you sho...",ULO1 - identify and analyse measurement system...
5,0ded927adb5e68102bdd077cd39619c4,ACB2120,2021.05,Financial accounting 2,FIN ACCT 2,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,This unit provides an overview of the current ...,Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S1-01-PENINSULA-ON-CAMPUS Offered-Y,�\n�,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr Lisa Powell,Responsible teaching Department of Accounting ...,ACB1120 OR ACC1100 OR ACF1100 OR ACW1120,NaN,ACF2100 OR ACW2120 OR ACC2100,Prerequisite: ACB1120 OR ACC1100 OR ACF1100 OR...,NaN,"On successful completion of this unit, you sho...","ULO1 - explain the content of, and regulatory ..."
6,7d4cc6a41b5aac10653b206b274bcb9a,ACB2220,2021.03RO,Management accounting 1,MGT ACCT 1,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,Introduction to management accounting. Topics ...,Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S1-01-PENINSULA-ON-CAMPUS Offered-Y,NaN,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr John Ko,Responsible teaching Department of Accounting ...,ACC1100 OR ACF1100 OR ACW1120 OR ACB1120,NaN,ACC2200 OR ACF2200 OR ACW2220,Prerequisite: ACC1100 OR ACF1100 OR ACW1120 OR...,NaN,"On successful completion of this unit, you sho...",ULO1 - describe cost behaviour under different...
7,b14cc6a41b5aac10653b206b274bcbbd,ACB2420,2021.04RO,Accounting information systems,ACC INFO SYS,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,"The objective of this unit is two-fold. First,...",Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S2-01-PENINSULA-ON-CAMPUS Offered-Y,NaN,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr Daisy Seng,Responsible teaching Department of Accounting ...,ACB1020 OR ACF1100 OR ACB1120 OR ACF1200 OR AC...,NaN,

### Get key columns

In [32]:
df_txt = df.loc[:, key_cols]

In [33]:
df_txt.isna().sum()

CL_UNIT_ID                 0
UNIT_CODE                  0
UNIT_TITLE                 0
HANDBOOK_SYNOPSIS         27
UNIT_LEARNING_OUTCOME    378
STUDY_LEVEL                2
OWNING_FACULTY             0
OWNING_ORG_UNIT            0
dtype: int64

In [34]:
df_txt.fillna("", inplace=True)

In [35]:
df_txt.isna().sum()

CL_UNIT_ID               0
UNIT_CODE                0
UNIT_TITLE               0
HANDBOOK_SYNOPSIS        0
UNIT_LEARNING_OUTCOME    0
STUDY_LEVEL              0
OWNING_FACULTY           0
OWNING_ORG_UNIT          0
dtype: int64

In [36]:
model_name = 'distilbert-base-nli-stsb-mean-tokens'
model = SentenceTransformer(model_name) # load DistillBERT model (more efficient) 

In [38]:
def get_bert_embeddings(text, model):
    """Computes the mean BERT embeddings (context dependent) for a given sentence
    Returns a 768 dimensional embedding
    """
    if text.strip() != "":
        embeddings = model.encode(text)
    else:
        embeddings = np.zeros(model.get_sentence_embedding_dimension())
    return embeddings

In [39]:
df_txt.loc[0, 'HANDBOOK_SYNOPSIS'], df_txt.loc[0, 'UNIT_LEARNING_OUTCOME']

('This unit introduces basic accounting concepts to non-accountants. The information requirements of two main groups of information users are examined - external users such as current and potential investors and internal users such as managers. This unit provides an introduction to the structure, meaning, analysis and interpretation of financial statements, in addition to exploring financial issues confronting managers, such as cost and performance measurement and budgeting.',
 'ULO1 - demonstrate an understanding of various forms of business organisations\nULO2 - apply financial and management accounting principles in the preparation of financial statements\nULO3 - measure and interpret information relating to financial performance, financial position, liquidity and risk indicators of businesses\nULO4 - measure and interpret financial and non-financial information for managers to use in planning, decision making and control\nULO5 - develop the ability to work effectively in a team and

### Baseline: Unit Synopsis

Few strategies:
- Combine embeddings (mean/sum) for both `HANDBOOK_SYNOPSIS` and `UNIT_LEARNING_OUTCOME`
- Keep them seperate to provide two distinct "modes" for viz that the users can select from
- Show them the top-K similar UNITS in terms of synopsis or outcomes

In [40]:
get_bert_embeddings(df_txt.loc[0, 'HANDBOOK_SYNOPSIS'], model).shape

(768,)

In [41]:
df_txt = df_txt.loc[df_txt.HANDBOOK_SYNOPSIS != "", :]
df_txt.shape

(5297, 8)

In [42]:
df_txt['embeddings'] = df_txt['HANDBOOK_SYNOPSIS'].apply(lambda x: get_bert_embeddings(x, model))

In [43]:
df_embeddings = pd.DataFrame(df_txt["embeddings"].to_list())

In [44]:
df_embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.155681,0.489606,-0.421600,-0.072158,-0.067547,0.274299,0.091877,-0.698768,-0.666343,-0.536237,0.291072,0.899176,-0.077659,-0.613539,0.343758,-0.670123,0.794246,-0.388422,0.641545,-0.456670,-0.119552,0.153932,0.466036,0.399631,-0.440171,-0.322607,0.073568,-0.297683,-0.059696,-1.148418,-0.071680,-0.730634,-0.144539,-0.316398,-0.140504,0.568030,0.627514,0.342634,-0.513060,0.573039,...,0.690227,0.179910,0.819151,-0.986035,-0.217857,0.704062,-0.557055,0.336980,0.073345,0.416183,0.169119,0.252819,-0.007765,0.867754,0.049940,0.622713,0.249839,0.490119,0.017128,0.472542,-0.081607,0.270919,-0.168286,0.216581,-0.160095,1.078925,0.140712,0.485378,-0.825715,-0.538115,-0.143330,-0.438859,-0.328312,0.467558,-0.623120,-0.232646,0.089066,0.373774,0.051465,0.133306
1,-0.131595,0.717931,0.103539,-0.352256,0.445363,0.574004,0.224938,-0.443664,0.156913,-0.621618,0.611865,0.595117,0.254392,-0.395437,0.282696,-0.721986,0.495153,-0.142491,0.425336,-0.847287,0.158907,-0.121272,-0.077499,0.322466,0.245117,0.059716,0.712812,0.279434,-0.062502,-0.594381,-0.161564,-0.912918,-0.276821,0.038880,0.144402,0.864495,0.575056,0.140458,-0.946866,0.566940,...,0.402116,0.236811,0.438252,-0.977629,-0.213731,1.024719,-0.511329,0.466470,0.013676,-0.338417,0.193394,-0.484011,-0.175995,1.359603,0.611458,-0.022046,0.306898,0.529888,-0.013254,0.200802,-0.388851,-0.016475,-0.287569,0.173027,0.000487,0.696105,0.154334,0.583121,-0.620628,-0.281843,-0.357997,-0.563932,-0.109533,0.522604,-0.882951,-0.611183,-0.418232,0.575920,-0.099176,0.193249
2,-0.088683,0.253432,-0.127657,0.622539,-0.325812,0.536132,0.729754,-0.175890,-0.649914,-0.419049,-0.334742,0.199344,0.924215,-0.092979,-0.252963,0.571394,0.298840,-0.037249,0.201663,-0.605967,-0.471863,-0.401052,0.603824,0.514850,1.017304,-0.253662,0.632144,1.323611,-0.037013,-0.371983,-0.155334,-0.673829,-0.292144,-0.339262,0.097976,0.239314,0.449920,0.534731,-1.278293,0.211376,...,0.944642,0.162069,0.349109,-1.309284,-0.536939,0.179826,0.029545,0.720614,0.231857,-0.428832,-0.070528,-0.317812,-0.905764,0.546344,0.519607,0.382819,-0.553631,0.094056,0.145244,0.248823,-0.619912,-0.656022,0.469421,-0.663645,-0.103491,0.573311,-0.210589,0.457057,-0.521667,-0.271598,-0.267051,-0.230766,-0.338813,1.029949,-0.810742,-0.761941,-0.479853,0.077615,0.325930,-0.238579
3,-0.087107,0.653962,-0.185667,-0.346532,0.271266,0.130659,0.110633,-0.481047,-0.453029,-0.704346,0.269104,0.765401,-0.131103,-0.477464,0.208585,-0.560700,0.167637,-0.375447,0.031502,-0.373972,0.472703,0.225208,0.008906,0.703815,-0.013692,0.098833,0.512104,0.422383,-0.036168,-0.866068,-0.052607,-1.309379,0.249422,-0.075759,-0.154235,0.623039,0.071654,0.113012,-1.043562,0.201496,...,0.398728,0.430202,0.548669,-1.208655,-0.139524,1.113375,0.092825,0.219635,0.400242,-0.533676,0.536091,0.274128,-0.216970,0.898456,0.493919,0.672126,0.379411,0.108186,-0.388592,0.055921,-0.419779,-0.068736,0.082193,-0.113954,-0.304841,0.383441,0.005209,0.868236,-0.421514,-0.385263,-0.639857,-0.303500,0.164779,0.129109,-0.683032,-0.802295,-0.375369,0.249854,-0.136509,0.575346
4,-0.532874,0.162291,0.077257,-0.460296,0.101121,-0.125485,0.050201,-0.356446,-0.598463,-1.237478,0.488874,0.532229,-0.091280,-0.702574,0.335763,-0.642425,0.524485,-0.317519,0.089133,-0.480850,0.275058,-0.027158,0.426448,0.599703,0.038397,-0.081412,0.517920,0.242777,0.089476,-0.650904,0.042536,-0.911383,-0.222596,-0.157123,0.055191,0.288849,-0.067637,0.022073,-0.545286,0.417792,...,0.471688,0.172623,0.781899,-1.341942,-0.104758,1.206891,0.046972,0.373366,0.314850,0.170563,0.918851,0.559490,0.105525,0.945548,0.284143,0.772073,-0.208892,-0.008863,-0.276362,0.641212,-0.104338,0.394144,-0.004151,0.318418,-0.228820,-0.008893,-0.172069,0.425589,-0.102983,-0.597770,-0

In [45]:
df_meta = df_txt.loc[:, ['UNIT_TITLE', 'UNIT_CODE', 'HANDBOOK_SYNOPSIS', 'STUDY_LEVEL', 'OWNING_FACULTY', 'OWNING_ORG_UNIT']]

In [46]:
df_meta.head()

,UNIT_TITLE,UNIT_CODE,HANDBOOK_SYNOPSIS,STUDY_LEVEL,OWNING_FACULTY,OWNING_ORG_UNIT
0,Accounting in business,ACB1020,This unit introduces basic accounting concepts...,undergraduate,Faculty of Business and Economics,Department of Accounting
2,Financial accounting 1,ACB1120,This unit provides you with an introduction to...,undergraduate,Faculty of Business and Economics,Department of Accounting
5,Financial accounting 2,ACB2120,This unit provides an overview of the current ...,undergraduate,Faculty of Business and Economics,Department of Accounting
6,Management accounting 1,ACB2220,Introduction to management accounting. Topics ...,undergraduate,Faculty of Business and Economics,Department of Accounting
7,Accounting information systems,ACB2420,"The objective of this unit is two-fold. First,...",undergraduate,Faculty of Business and Economics,Department of Accounting


Remove newlines

In [47]:
df_meta['HANDBOOK_SYNOPSIS'] = df_meta.HANDBOOK_SYNOPSIS.apply(lambda x: x.replace("\n", " "))

### Export results

In [ ]:
df_txt.to_csv("unit_synopsis_embeddings.csv", index=False)

In [48]:
df_embeddings.to_csv("/content/drive/MyDrive/knowledge-mapper/data/unit_synopsis_embeddings_only.tsv", sep="\t", header=False, index=False)

In [49]:
df_meta.to_csv("/content/drive/MyDrive/knowledge-mapper/data/unit_synopsis_metadata_only.tsv", sep="\t", index=False)

### Baseline: Learning Outcome

Filtering/cleaning

In [107]:
import re
pattern = r'ULO[\d]+[\s][\-][\s]?'

In [109]:
def apply_filter(x):
    x = re.sub(pattern, '', x)
    x = x.replace('\n', '. ')
    return x

In [110]:
df_txt['UNIT_LEARNING_OUTCOME_TIDY'] = df_txt['UNIT_LEARNING_OUTCOME'].apply(lambda x: apply_filter(x))

In [111]:
df_txt.UNIT_LEARNING_OUTCOME_TIDY

0       demonstrate an understanding of various forms ...
2       identify and analyse measurement systems and t...
5       explain the content of, and regulatory require...
6       describe cost behaviour under different assump...
7       examine the role of accounting information sys...
                              ...                        
5843    Demonstrate self-motivation, and able to const...
5844    Justify your conceptual, material and logistic...
5845    Compose a self-directed and coherent work plan...
5846                                                     
5847                                                     
Name: UNIT_LEARNING_OUTCOME_TIDY, Length: 5324, dtype: object

In [117]:
df_txt = df_txt.loc[df_txt.UNIT_LEARNING_OUTCOME_TIDY != "", :]

In [118]:
df_txt.shape

(4946, 10)

In [119]:
df_txt['embeddings'] = df_txt['UNIT_LEARNING_OUTCOME_TIDY'].apply(lambda x: get_bert_embeddings(x, model))

In [120]:
df_embeddings = pd.DataFrame(df_txt["embeddings"].to_list())

In [121]:
df_embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.190574,0.783615,-0.142525,-0.069620,0.628788,0.299740,0.301032,-0.620602,-0.489804,-0.967707,0.097668,0.593581,0.025283,-0.501025,-0.243577,-0.446971,0.731702,-0.217442,0.126845,-0.334100,0.245078,-0.151419,0.165306,0.303316,0.197842,-0.343014,0.121667,0.480170,0.383156,-1.183235,-0.112575,-1.236977,-0.025756,-0.259758,-0.235255,0.589923,0.793278,0.018879,-1.403651,0.220573,...,0.908003,0.187791,0.541992,-1.191732,-0.262007,0.418833,-0.334067,0.442791,-0.510821,-0.218818,0.806002,0.211932,-0.018386,0.687508,-0.296969,-0.144197,0.180340,0.619776,-0.578515,0.698252,0.155741,-0.599011,0.038766,-0.176376,-0.027731,0.793048,0.132691,1.151903,-0.818680,-0.555997,-0.249554,-0.548605,0.147593,0.386955,-0.316573,-0.309027,-0.167690,-0.014741,0.206307,-0.193014
1,0.442342,0.663459,-0.040303,0.182033,0.287935,0.396413,0.502220,-0.562374,0.086399,-0.736350,0.212181,0.298762,-0.007652,-0.370564,-0.116598,-0.187309,0.337287,-0.267863,0.239586,-0.423995,0.268260,0.065869,0.163722,0.230965,0.275066,0.033609,0.508122,0.650827,-0.065164,-1.050356,0.221013,-1.343909,-0.277504,0.138498,0.015265,0.757364,0.691240,-0.272044,-1.184375,0.308476,...,0.638602,0.236545,0.598251,-1.140565,-0.299819,0.283043,0.021340,0.041987,-0.061698,-0.296405,0.564465,0.296288,0.052979,0.768949,0.182334,-0.294449,0.092165,0.201847,-0.123586,0.538418,-0.120854,-0.065711,-0.005356,-0.079292,0.014524,0.809220,0.118108,0.518450,-0.570156,-0.514727,-0.215204,-0.384749,0.221199,0.832066,-0.588055,-0.584953,-0.057417,0.153753,0.193183,-0.178473
2,0.182996,0.567996,-0.228919,-0.012939,0.203105,0.118654,0.115995,-0.396435,-0.730736,-0.540764,0.242228,0.430015,-0.172629,-0.467906,-0.168219,-0.161378,0.540848,-0.318584,0.085767,-0.624560,0.239795,0.171395,0.117106,0.218151,0.261276,-0.109520,0.231926,0.370106,0.095174,-1.031990,-0.123283,-0.750224,-0.178646,-0.080195,-0.206044,0.802806,0.267035,-0.084177,-1.385462,0.070352,...,0.336421,0.106981,0.526665,-0.862007,0.225086,0.563124,-0.546155,0.195700,-0.410705,-0.255296,0.411470,-0.299267,0.172670,0.511499,-0.144807,0.361173,0.029052,0.257695,0.006258,0.464535,-0.054838,-0.304158,0.513952,-0.422245,-0.182104,0.697651,0.282732,0.901309,-0.771497,-0.522944,0.090704,-0.136751,0.043783,0.898061,-0.288495,-0.701310,-0.090514,0.245890,0.300523,-0.239964
3,-0.244238,0.529993,-0.264925,0.513825,0.237940,0.629372,0.021398,-0.425021,0.052999,-0.962412,0.349542,1.118321,-0.267314,-0.598368,-0.253053,0.297845,0.510975,-0.204439,-0.475494,-0.383782,0.487716,0.095708,-0.037705,0.515036,0.254094,-0.009826,0.242509,0.587813,0.122601,-0.761763,-0.141404,-0.464770,-0.120359,0.109065,-0.141289,0.346288,0.397101,-0.646319,-0.525161,-0.099231,...,0.280768,-0.029017,0.858594,-0.950786,0.197694,0.513545,-0.096666,0.393268,0.066020,-0.182446,0.308711,0.189645,-0.328527,0.287888,0.549762,0.148189,-0.285872,0.233558,-0.092933,0.525911,-0.222348,-0.428224,0.343439,-0.346770,-0.436319,0.603467,0.406047,0.744379,-0.435040,-0.039783,-0.852077,-0.610677,0.097273,0.248185,-0.670033,-0.017029,-0.087662,0.142614,0.034854,0.007149
4,-0.217312,0.807257,0.005327,-0.229301,0.487162,0.437366,0.124242,-0.498675,-0.795689,-0.659403,0.245554,0.694550,-0.234083,-0.749190,-0.206064,-0.657972,0.409081,-0.036519,0.220769,-0.559688,0.489801,0.030136,0.523244,0.650125,0.180438,-0.236281,0.211935,-0.080460,-0.165211,-1.095602,-0.349733,-1.138006,-0.152815,0.131041,-0.294718,0.835831,0.650029,0.290475,-1.074150,0.343879,...,0.444755,0.066284,0.631859,-1.004398,-0.052123,1.123194,-0.213117,-0.084535,-0.239304,-0.175036,0.442843,-0.041634,-0.133257,0.965774,0.307732,0.179432,-0.071480,0.547723,-0.047101,0.324160,-0.141484,-0.116053,0.060186,-0.035036,0.067235,0.814690,0.110841,0.615606,-0.723030,-0.687181,-

In [124]:
df_meta = df_txt.loc[:, ['UNIT_TITLE', 'UNIT_CODE', 'UNIT_LEARNING_OUTCOME', 'STUDY_LEVEL', 'OWNING_FACULTY', 'OWNING_ORG_UNIT']]

In [125]:
df_meta.head()

,UNIT_TITLE,UNIT_CODE,UNIT_LEARNING_OUTCOME,STUDY_LEVEL,OWNING_FACULTY,OWNING_ORG_UNIT
0,Accounting in business,ACB1020,ULO1 - demonstrate an understanding of various...,undergraduate,Faculty of Business and Economics,Department of Accounting
2,Financial accounting 1,ACB1120,ULO1 - identify and analyse measurement system...,undergraduate,Faculty of Business and Economics,Department of Accounting
5,Financial accounting 2,ACB2120,"ULO1 - explain the content of, and regulatory ...",undergraduate,Faculty of Business and Economics,Department of Accounting
6,Management accounting 1,ACB2220,ULO1 - describe cost behaviour under different...,undergraduate,Faculty of Business and Economics,Department of Accounting
7,Accounting information systems,ACB2420,ULO1 - examine the role of accounting informat...,undergraduate,Faculty of Business and Economics,Department of Accounting


Remove newlines (?)

In [128]:
df_meta['UNIT_LEARNING_OUTCOME'] = df_meta.UNIT_LEARNING_OUTCOME.apply(lambda x: x.replace("\n", " "))

### Export results

In [ ]:
df_txt.to_csv("unit_learning_outcomes_embeddings.csv", index=False)

In [126]:
df_embeddings.to_csv("unit_learning_outcomes_embeddings_only.tsv", sep="\t", header=False, index=False)

In [130]:
df_meta.to_csv("unit_learning_outcomes_metadata_only.tsv", sep="\t", index=False)

### UMAP Coordinates

In [4]:
!ls ../embedding-projector-standalone/oss_data/umap_coord

bnmt_enjako_interlingua_10k_1024_bookmarks.txt
bnmt_enjako_interlingua_10k_1024_metadata.tsv
bnmt_enjako_interlingua_10k_1024_tensors.bytes
iris_labels.tsv
iris_tensors.bytes
knowledge_mapper_config.json
mnist_10k_784d_labels.tsv
mnist_10k_784d_tensors.bytes
mnist_10k_sprite.png
oss_demo_projector_config.json
processed
umap_coord
word2vec_10000_200d_labels.tsv
word2vec_10000_200d_tensors.bytes
word2vec_full_200d_labels.tsv
word2vec_full_200d_tensors.bytes
word2vec_full_bookmarks.txt


In [8]:
import json

In [10]:
with open('../embedding-projector-standalone/oss_data/umap_coord', 'r') as f:
    umap = json.load(f)

In [12]:
umap[:5]

[[-0.43100563803496783, -1.8316399139264095, 1.4080871191658622],
 [-2.087463554599129, 2.848090584938719, -1.128744903378506],
 [-0.29089829754151286, -1.0699792166318993, 2.6348052971111726],
 [-1.4488307949959374, -0.4281773898773557, 0.3497297321284385],
 [-1.234321606816859, -1.3937491172873546, 1.4808987771586961]]

EOF